In [1]:
import pickle
with open('train_sentence_transformer.pkl', 'rb') as f:
    train = pickle.load(f)
with open('val_sentence_transformer.pkl', 'rb') as f:
    val = pickle.load(f)
with open('test_sentence_transformer.pkl', 'rb') as f:
    test = pickle.load(f)

In [2]:
import json
with open('test.json', encoding = 'utf8') as json_file:
    data = json.load(json_file)
    
conversations = []
summaries = []
for i in range(0, len(data)):
    if len(data[i]['dialogue'].split('\r\n')) > 1:
        sentences = data[i]['dialogue'].split('\r\n')
    else:
        sentences = data[i]['dialogue'].split('\n')

    conversations.append(sentences)
    summaries.append(data[i]['summary'].strip('\n').replace('\r\nt', ' '))

In [3]:
import numpy as np
length = []
for i in range(0, len(train)):
    length.append(len(train[i]))
for i in range(0, len(val)):
    length.append(len(val[i]))
for i in range(0, len(test)):
    length.append(len(test[i]))
X = []
for i in range(0, len(train)):
    for j in range(0, len(train[i])):
        X.append(np.array(train[i][j]))
for i in range(0, len(val)):
    for j in range(0,len(val[i])):
        X.append(np.array(val[i][j]))
for i in range(0, len(test)):
    for j in range(0, len(test[i])):
        X.append(np.array(test[i][j]))

In [4]:
from hmmlearn import hmm

In [5]:
remodel = hmm.GaussianHMM(n_components=4, n_iter = 50, covariance_type = 'diag', verbose = True, init_params="cm", params="cmts")
remodel.startprob_ = np.array([1, 0.0, 0.0, 0.0])
remodel.transmat_ = np.array([
                 [0.33, 0.34, 0.33, 0],
                 [0.0, 0.33, 0.34, 0.33],
                 [0.0, 0.0, 0.5, 0.5],
                 [0.0, 0.0, 0.0, 1.0]]
                            )

In [6]:
remodel.startprob_

array([1., 0., 0., 0.])

In [7]:
remodel.transmat_

array([[0.33, 0.34, 0.33, 0.  ],
       [0.  , 0.33, 0.34, 0.33],
       [0.  , 0.  , 0.5 , 0.5 ],
       [0.  , 0.  , 0.  , 1.  ]])

In [8]:
remodel.fit(X, length)

         1 -824497.35069695             +nan
         2 -800119.10664105  +24378.24405590
         3 -798716.12516782   +1402.98147323
         4 -797928.19256250    +787.93260532
         5 -797369.29833655    +558.89422595
         6 -796682.67659204    +686.62174451
         7 -796083.43651333    +599.24007870
         8 -795434.51166870    +648.92484464
         9 -794934.40531554    +500.10635315
        10 -794580.41982900    +353.98548654
        11 -794350.57728535    +229.84254365
        12 -793914.51086257    +436.06642278
        13 -793598.11274955    +316.39811302
        14 -793496.62716232    +101.48558722
        15 -793234.09115373    +262.53600859
        16 -792969.92150603    +264.16964770
        17 -792812.95859511    +156.96291092
        18 -792758.54048191     +54.41811320
        19 -792739.98567726     +18.55480465
        20 -792709.35655127     +30.62912598
        21 -792699.49403690      +9.86251438
        22 -792697.68464553      +1.80939137
        23

GaussianHMM(init_params='cm', n_components=4, n_iter=50, params='cmts',
            verbose=True)

In [9]:
remodel.transmat_

array([[0.75623402, 0.16037183, 0.08339415, 0.        ],
       [0.        , 0.82055435, 0.14513559, 0.03431006],
       [0.        , 0.        , 0.88489168, 0.11510832],
       [0.        , 0.        , 0.        , 1.        ]])

In [10]:
remodel.startprob_

array([1., 0., 0., 0.])

In [11]:
def encode_convs(profix):
    sent_label = []
    with open(profix + '_sentence_transformer.pkl', 'rb') as f:
        data = pickle.load(f)
    for i in range(0, len(data)):
   
        labels = remodel.decode(np.array(data[i]))[1]
        sent_label.append(labels)
    
    with open(profix + '_sent_trans_cons_label.pkl', 'wb') as f:
        pickle.dump(sent_label, f)
    return sent_label

In [12]:
l = encode_convs('train')

In [13]:
l = encode_convs('val')

In [14]:
l = encode_convs('test')